In [21]:
!pip install rouge_score
# from rouge_score import rouge_scorer
# from nltk.translate.meteor_score import meteor_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=964a5c7b73ffcd55f349aa6b30e78ccc303891c762edca9f8de146b630f3d188
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import torch
import json
import nltk
# import sacrebleu
from rouge_score import rouge_scorer
# from nltk.translate.meteor_score import meteor_score
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "tadkt/Qwen-2.5-0.5B-VN/checkpoint-500-s2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


The repository for tadkt/Qwen-2.5-0.5B-VN contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/tadkt/Qwen-2.5-0.5B-VN.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(166473, 896, padding_idx=151643)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (n

In [23]:
def load_jsonl(filename):
    with open(filename, "r", encoding="utf-8") as f:
        return [json.loads(line) for line in f]

data = load_jsonl("/kaggle/input/val-jsonl/val_dataset-1.jsonl")

test_data = []
for entry in data:
    user_prompt = None
    expected_output = None

    for message in entry["messages"]:
        if message["role"] == "user":
            user_prompt = message["content"]
        elif message["role"] == "assistant":
            expected_output = message["content"]

    if user_prompt and expected_output:
        test_data.append({"input": user_prompt, "expected_output": expected_output})

# test_data

In [30]:
def generate_response(text, max_new_tokens = 50):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    inputs.pop("token_type_ids", None) 

    with torch.no_grad():
        output = model.generate(**inputs,  max_new_tokens=max_new_tokens)

    return tokenizer.decode(output[0], skip_special_tokens=False)

for entry in test_data:
    entry["predicted_output"] = generate_response(entry["input"], max_new_tokens=100)


KeyboardInterrupt: 

In [15]:
def evaluate_metrics(data):
    bleu_scores, rouge_scores, meteor_scores = [], [], []

    rouge = rouge_scorer.RougeScorer(["rouge-1", "rouge-2", "rouge-l"], use_stemmer=True)

    for entry in data:
        reference = entry["expected_output"]
        prediction = entry["predicted_output"]

        bleu = sacrebleu.sentence_bleu(prediction, [reference])
        bleu_scores.append(bleu.score)

        rouge_result = rouge.score(prediction, reference)
        rouge_scores.append(rouge_result["rouge-l"].fmeasure)

        # # METEOR Score
        # meteor = meteor_score([reference], prediction)
        # meteor_scores.append(meteor)

    print(f"BLEU Score: {sum(bleu_scores) / len(bleu_scores):.4f}")
    print(f"ROUGE-L Score: {sum(rouge_scores) / len(rouge_scores):.4f}")
    # print(f"METEOR Score: {sum(meteor_scores) / len(meteor_scores):.4f}")

evaluate_metrics(test_data)


Average ROUGE-1 F1: 0.1662
Average ROUGE-2 F1: 0.0431
Average ROUGE-L F1: 0.1430
